In [1]:
from pyspark.sql import SparkSession

try:
    # Tworzenie sesji Spark
    spark = SparkSession.builder \
        .appName("PySpark SQL Server Connection") \
        .config("spark.jars", "C:\\Projekty\\Wielowymiarowa\\Wielowymiarowa_analiza_danych\\sqljdbc_12.6\\enu\\jars\\mssql-jdbc-12.6.1.jre8.jar") \
        .getOrCreate()

    # Parametry połączenia z bazą danych MSSQL
    server_name = "localhost"
    port = "1433"
    database_name = "NowaBazaDanych2"
    url = f"jdbc:sqlserver://{server_name}:{port};databaseName={database_name}"

    table_name = "Pracownicy15"
    username = "sa"
    password = "YourStrongPassword123"

    # Wczytanie danych z bazy danych MSSQL
    df = spark.read \
        .format("jdbc") \
        .option("url", url) \
        .option("dbtable", table_name) \
        .option("user", username) \
        .option("password", password) \
        .option("encrypt", "false") \
        .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
        .load()

    print("Dane zostały pomyślnie wczytane z MSSQL.")
    # Wyświetlenie pierwszych kilku wierszy DataFrame
    df.show()

except Exception as e:
    print("Wystąpił błąd podczas łączenia z bazą danych:", str(e))



Dane zostały pomyślnie wczytane z MSSQL.
+---------+--------------------+--------------------+--------+----+-----------+-----------+--------+------+--------------------+--------------------+--------+--------+------------+---------+--------------------+--------------+--------------------+------+------------+--------+--------+--------+--------+--------------------+--------------------+-------+---------+
|    DR_NO|           Date Rptd|            DATE OCC|TIME OCC|AREA|  AREA NAME|Rpt Dist No|Part 1-2|Crm Cd|         Crm Cd Desc|             Mocodes|Vict Age|Vict Sex|Vict Descent|Premis Cd|         Premis Desc|Weapon Used Cd|         Weapon Desc|Status| Status Desc|Crm Cd 1|Crm Cd 2|Crm Cd 3|Crm Cd 4|            LOCATION|        Cross Street|    LAT|      LON|
+---------+--------------------+--------------------+--------+----+-----------+-----------+--------+------+--------------------+--------------------+--------+--------+------------+---------+--------------------+--------------+-----

In [2]:
df.show(10)

+---------+--------------------+--------------------+--------+----+-----------+-----------+--------+------+--------------------+-------------------+--------+--------+------------+---------+--------------------+--------------+--------------------+------+-----------+--------+--------+--------+--------+--------------------+--------------------+-------+---------+
|    DR_NO|           Date Rptd|            DATE OCC|TIME OCC|AREA|  AREA NAME|Rpt Dist No|Part 1-2|Crm Cd|         Crm Cd Desc|            Mocodes|Vict Age|Vict Sex|Vict Descent|Premis Cd|         Premis Desc|Weapon Used Cd|         Weapon Desc|Status|Status Desc|Crm Cd 1|Crm Cd 2|Crm Cd 3|Crm Cd 4|            LOCATION|        Cross Street|    LAT|      LON|
+---------+--------------------+--------------------+--------+----+-----------+-----------+--------+------+--------------------+-------------------+--------+--------+------------+---------+--------------------+--------------+--------------------+------+-----------+--------+--

In [3]:

# Zakończenie sesji Spark
spark.stop()

In [3]:
from pyspark.sql import SparkSession
import pandas as pd

# Inicjalizacja sesji Spark
spark = SparkSession.builder \
    .appName("Adding Data from CSV to PostgreSQL") \
    .config("spark.driver.extraClassPath", "postgresql-42.7.2.jar") \
    .config("spark.executor.extraClassPath", "postgresql-42.7.2.jar") \
    .getOrCreate()

# Wczytaj dane z pliku CSV za pomocą Pandas
pandas_df = pd.read_csv("latlon_RZ.csv", sep=",", dtype=str)

pandas_df = pandas_df.drop('Unnamed: 0', axis=1)

# Konwersja ramki danych Pandas na ra
# mkę danych Sparka
df = spark.createDataFrame(pandas_df)

# Zapis danych do bazy danych PostgreSQL
df.write \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://localhost:5432/my_database") \
    .option("dbtable", "table_name5") \
    .option("user", "my_user") \
    .option("password", "123") \
    .mode("append") \
    .save()

# Zakończenie sesji Spark
spark.stop()


Py4JJavaError: An error occurred while calling o99.save.
: java.sql.SQLException: No suitable driver
	at java.sql/java.sql.DriverManager.getDriver(DriverManager.java:300)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.$anonfun$driverClass$2(JDBCOptions.scala:109)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.<init>(JDBCOptions.scala:109)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcOptionsInWrite.<init>(JDBCOptions.scala:254)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcOptionsInWrite.<init>(JDBCOptions.scala:258)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcRelationProvider.createRelation(JdbcRelationProvider.scala:47)
	at org.apache.spark.sql.execution.datasources.SaveIntoDataSourceCommand.run(SaveIntoDataSourceCommand.scala:48)
	at org.apache.spark.sql.execution.command.ExecutedCommandExec.sideEffectResult$lzycompute(commands.scala:75)
	at org.apache.spark.sql.e